<a href="https://colab.research.google.com/github/Umanginigam/Langgraph/blob/main/Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries for building the agentic workflow.
# langgraph: workflow engine for agents
# langchain: abstraction for LLMs and tools
# transformers: for open-source language models
# accelerate: optimized hardware usage
!pip install --quiet langgraph langchain transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
from transformers import pipeline

llm = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=256)



Device set to use cpu


In [ ]:
def plan_agent_node(state):
    # Check if there are existing tasks using Pydantic-style access
    if not state.tasks:
        # Initial planning from user query
        prompt = f"Break this user request into numbered subtasks:\nUser query: {state.user_query}"
    else:
        # Refine existing tasks based on feedback
        prompt = f"Refine these subtasks based on the feedback:\nTasks: {state.tasks}\nFeedback: {state.feedback}"

    # Call the language model (LLM) to generate tasks
    response = llm(prompt)[0]['generated_text']

    # Split response into a clean list of tasks
    tasks = [line.strip() for line in response.split('\n') if line.strip()]

    # Return a partial state update (LangGraph will merge this into the full state)
    return {
        "tasks": tasks,
        "feedback": []  # Reset feedback after planning
    }



In [ ]:
def tool_agent_node(state):
    results = []

    # ✅ Loop through each subtask and generate step-by-step solutions
    for task in state.tasks:
        # Construct the prompt for the LLM
        prompt = f"Step-by-step solution for: {task}"

        # Call the language model (assuming llm returns a list of completions)
        response = llm(prompt)[0]['generated_text']

        # Collect the generated result
        results.append(response)

    # ✅ Return a dictionary that matches the expected keys in the state schema
    return {
        "results": results
    }


In [ ]:
def reflection_node(state):
    feedback = []

    # ✅ Access results using dot notation
    for result in state.results:
        # Prompt the LLM to evaluate each result and provide feedback
        prompt = f"Evaluate this result and suggest improvement if needed:\n{result}"
        fb = llm(prompt)[0]['generated_text']
        feedback.append(fb)

    # ✅ Return the updated feedback as expected by LangGraph
    return {
        "feedback": feedback
    }


In [ ]:
def needs_more_refinement(feedback: list) -> str:
    for f in feedback:
        if "improve" in f.lower() or "not clear" in f.lower():
            return "continue"
    return "END"


In [ ]:
from langgraph.graph import StateGraph, END
from pydantic import BaseModel
from typing import List

# ✅ Define schema using Pydantic (not a plain dict)
class AgentState(BaseModel):
    user_query: str
    tasks: List[str] = []
    results: List[str] = []
    feedback: List[str] = []

# ✅ Build graph using AgentState
graph = StateGraph(AgentState)

# Add nodes and edges as before
graph.add_node("PlanAgent", plan_agent_node)
graph.add_node("ToolAgent", tool_agent_node)
graph.add_node("Reflection", reflection_node)

graph.set_entry_point("PlanAgent")
graph.add_edge("PlanAgent", "ToolAgent")
graph.add_edge("ToolAgent", "Reflection")

# Conditional edge based on feedback
graph.add_conditional_edges(
    "Reflection",
    lambda state: needs_more_refinement(state.feedback),
    {
        "continue": "PlanAgent",
        "END": END
    }
)

# Compile the graph
app = graph.compile()


In [ ]:
initial_state = AgentState(
    user_query="Generate a data report for sales in 2024 and visualize trends using Python"
)

final_state = app.invoke(initial_state)

# 👇 Add this to see the output
print("🧠 Final Agent State:\n")
print(final_state)


🧠 Final Agent State:

{'user_query': 'Generate a data report for sales in 2024 and visualize trends using Python', 'tasks': ['Data report for sales in 2024 and visualize trends using Python'], 'results': ['Solution: Data report for sales in 2024 and visualize trends using Python'], 'feedback': ['This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This results in a 32% increase in sales in 2024 compared to the same period last year. This re